In [1]:
# pytorch optimizer 让动量参与计算，以及手动修改lr

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np

import ray
import resnet.models as models
import random,time
from time import sleep
import copy 
import datetime
import argparse


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np
import os
import shutil
from torch.utils.tensorboard import SummaryWriter
from filelock import FileLock

/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [2]:
model1 = models.__dict__["resnet"](dataset="cifar10",depth=164)
optimizer1 = optim.SGD(model1.parameters(),
                          lr=0.1,
                          momentum=0.9,
                          weight_decay=1e-4)
model2 = models.__dict__["resnet"](dataset="cifar10",depth=164)
optimizer2 = optim.SGD(model2.parameters(),
                          lr=100,
                          momentum=0.9,
                          weight_decay=1e-4)

In [3]:
def generate_train_loader(batch_size,kwargs):
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=True, download=True,
        transform=transforms.Compose([
            transforms.Pad(4),
            transforms.RandomCrop(32),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
        ])),
    batch_size=batch_size, shuffle=True, **kwargs)
    return train_loader
train_loader = generate_train_loader(64,{})

Files already downloaded and verified


In [4]:
batch_idx,(data,target) =next(enumerate(train_loader))
data, target = Variable(data), Variable(target)
optimizer1.zero_grad()
output = model1(data)
loss1 = F.cross_entropy(output,target)
pred = output.data.max(1,keepdim=True)[1]
batch_acc = pred.eq(target.data.view_as(pred)).cpu().sum()
old_tensor = copy.deepcopy([p.data for p in model1.parameters()])
loss1.backward()
old_tensor2 = copy.deepcopy([p.data for p in model1.parameters()])
optimizer1.step()
new_tensor = [p.data for p in model1.parameters()]

In [7]:
old_tensor[0] ==  old_tensor2[0]

tensor([[[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]],


        [[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]],


        [[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]],


        [[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]],


        [[[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]],

         [[1, 1, 1],
          [1, 1, 1],
          [1, 1, 1]]],


        [[[1, 1

tensor([[[[ 6.5117e-02, -6.8695e-02,  2.2021e-01],
          [ 3.1654e-01, -2.3344e-01, -8.8615e-02],
          [ 7.6554e-03,  1.6118e-01, -6.7293e-02]],

         [[ 2.1374e-02,  1.1244e-01,  1.6685e-01],
          [ 1.7423e-01, -1.0647e-01, -1.1521e-01],
          [ 1.1857e-01,  1.7958e-01, -1.5464e-01]],

         [[-1.9924e-02,  1.5005e-01,  4.4446e-02],
          [ 7.2835e-02, -1.2540e-01,  2.4413e-01],
          [-8.6074e-02,  5.7862e-02, -6.0246e-02]]],


        [[[-2.0736e-01, -3.8466e-02, -2.1230e-01],
          [ 1.3637e-01, -5.0587e-02, -2.4874e-01],
          [ 8.0520e-03,  3.0126e-02, -5.5083e-03]],

         [[-5.6765e-02,  3.8965e-02,  1.2322e-01],
          [ 4.1526e-02,  4.0742e-02,  2.8007e-02],
          [-1.1020e-01, -4.7171e-02,  3.9003e-02]],

         [[-6.5864e-02, -8.2997e-02, -1.7644e-01],
          [ 1.3888e-01,  1.6684e-02, -2.2989e-02],
          [-1.8378e-01,  1.4718e-01,  1.1404e-01]]],


        [[[-1.1641e-01, -1.4976e-02,  2.5454e-01],
          [ 1.5

In [ ]:
diff = [old_tens - new_tens for (old_tens, new_tens) in zip(old_tensor,new_tensor)]

In [ ]:
diff[0]

In [ ]:
[1,2,3] - [4,5,6]